In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
physical_devices = tf.config.list_physical_devices('GPU') 
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

%load_ext tensorboard
import datetime

tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
!rm -rf ./logs/LSTM/ #should not run this line it will remove the tensorboard logs

In [3]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [4]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [5]:
#retrieve and format data - into labels and examples from the dataset
def features_and_labels(filename_a, filename_b):
    with open(filename_b, 'rb') as f:
        labels = np.load(f).ravel()
    
    data = pd.read_pickle(filename_a)
    #https://stackoverflow.com/questions/22798934/pandas-long-to-wide-reshape-by-two-variables
    data_x = data.pivot(index='id', columns='time', values='FLUX')
    data_x = data_x.to_numpy()
    data_x = convert_3D(normalize(data_x))
    return data_x, labels

In [6]:
training_data_x, training_data_y = features_and_labels("../Kepler-Train.pkl", "../Labels-Train.npy")
testing_data_x, testing_data_y = features_and_labels("../Kepler-Test.pkl", "../Labels-Test.npy")
validation_data_x, validation_data_y = features_and_labels("../Kepler-Val.pkl", "../Labels-Val.npy")

In [7]:
LSTMmodel = Sequential([
    LSTM(64, input_shape=(training_data_x.shape[1:]), return_sequences=True),
    #Dropout(0.3),
    LSTM(64),
    #Dropout(0.3),
    Dense(1, activation='sigmoid')
])

LSTMmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.000003),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/LSTM/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasLSTM/training.csv',separator=",", append=False)

LSTMmodel.fit(training_data_x, training_data_y, epochs=10, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = LSTMmodel
model.save('keras-models/KerasLSTM')

Epoch 1/10
394/394 [==============================] - 14s 23ms/step - loss: 0.6876 - accuracy: 0.5528 - auc: 0.5130 - precision: 0.1451 - recall: 0.1944 - false_negatives: 2325.0000 - false_positives: 3305.0000 - true_negatives: 6398.0000 - true_positives: 561.0000 - val_loss: 0.6821 - val_accuracy: 0.5978 - val_auc: 0.5387 - val_precision: 0.0696 - val_recall: 0.0609 - val_false_negatives: 339.0000 - val_false_positives: 294.0000 - val_true_negatives: 919.0000 - val_true_positives: 22.0000
Epoch 2/10
394/394 [==============================] - 7s 18ms/step - loss: 0.6767 - accuracy: 0.6961 - auc: 0.5689 - precision: 0.0374 - recall: 0.0132 - false_negatives: 2848.0000 - false_positives: 978.0000 - true_negatives: 8725.0000 - true_positives: 38.0000 - val_loss: 0.6698 - val_accuracy: 0.7706 - val_auc: 0.6396 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_false_negatives: 361.0000 - val_false_positives: 0.0000e+00 - val_true_negatives: 1213.0000 - val_true_positives: 0.0000e+

INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


INFO:tensorflow:Assets written to: keras-models/KerasLSTM/assets


In [8]:
%tensorboard --logdir logs/LSTM/fit